평가 개요
- 교과목: 빅데이터 시스템 개발
- 능력단위: 빅데이터 처리시스템 개발
- 훈련교사명: 정지훈 강사
- 수강생명: 권도근
- 평가방법: 기타(작업형)
- 평가예정일: 2022년 02월 14일 월요일 (1시간)

평가 문항
- 두개의 데이터 파일을 불러와서 각 영화별 얼마나 자주 방영되었는지 숫자를 세는 코드를 작성하세요. 
    + 단, PySpark 문법으로 파일을 작성합니다. 

In [1]:
# (1) 필수 라이브러리 설치
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import StructType, StructField, IntegerType, LongType
import codecs
# (2) u.item 함수 정의 만들기
def loadNames():
    Names = {}
# (3) 데이터 불러오는 코드 작성
    with codecs.open("u.item", "r",
                     encoding='ISO-8859-1',
# (4) 정확한 인코딩 표기 작성
                     errors='ignore') as f:
        for line in f:
            fields = line.split('|')
            Names[int(fields[0])] = fields[1]
    return Names
def main():
    # (5) spark 세션 할당
    spark = SparkSession.builder.appName("PopularMovies").getOrCreate()
    nameDict = spark.sparkContext.broadcast(loadNames())
    # (6) u.logs 파일에 대한 스키마 작성
    schema = StructType([ \
                         StructField("userID", IntegerType(), True), \
                         StructField("movieID", IntegerType(), True), \
                         StructField("rating", IntegerType(), True), \
                         StructField("time", LongType(), True)])
    # (7) log파일을 데이터 프레임 형태로 불러오기
    moviesDF = spark.read.option("sep", "\t").schema(schema).csv("u.logs")
    movieCounts = moviesDF.groupBy("movieID").count()
    def lookupName(movieID):
        return nameDict.value[movieID]
    lookupNameUDF = func.udf(lookupName)
    # (8) movieTitle 컬럼 추가
    moviesWithNames = movieCounts.withColumn("movieTitle", lookupNameUDF(func.col("movieID")))
    # (9) 결과 정렬 및 상위 20개 추출
    sortedMoviesWithNames = moviesWithNames.orderBy(func.desc("count"))
    sortedMoviesWithNames.withColumn("Creator", func.lit("권도근")).show(20, False)
    # 세션 종료
    spark.stop()
if __name__ == "__main__":
    main()

+-------+-----+--------------------------------+-------+
|movieID|count|movieTitle                      |Creator|
+-------+-----+--------------------------------+-------+
|50     |583  |Star Wars (1977)                |권도근 |
|258    |509  |Contact (1997)                  |권도근 |
|100    |508  |Fargo (1996)                    |권도근 |
|181    |507  |Return of the Jedi (1983)       |권도근 |
|294    |485  |Liar Liar (1997)                |권도근 |
|286    |481  |English Patient, The (1996)     |권도근 |
|288    |478  |Scream (1996)                   |권도근 |
|1      |452  |Toy Story (1995)                |권도근 |
|300    |431  |Air Force One (1997)            |권도근 |
|121    |429  |Independence Day (ID4) (1996)   |권도근 |
|174    |420  |Raiders of the Lost Ark (1981)  |권도근 |
|127    |413  |Godfather, The (1972)           |권도근 |
|56     |394  |Pulp Fiction (1994)             |권도근 |
|7      |392  |Twelve Monkeys (1995)           |권도근 |
|98     |390  |Silence of the Lambs, The (1991)|권도근 |
|237    |384  |Jerr